In [1]:
import pandas as pd
from pathlib import Path
from astropy.time import Time
from mirar.data import SourceBatch, SourceTable, Dataset
from mirar.pipelines import get_pipeline
from mirar.pipelines.winter.load_winter_image import sncosmo_filters
from mirar.database.constraints import DBQueryConstraints
from mirar.database.transactions.update import _update_database_entry
from mirar.database.transactions import select_from_table
from mirar.pipelines.winter.models import Diff
import logging
import numpy as np
from tqdm import tqdm
from astropy import units as u
from astropy.coordinates import SkyCoord
from winterdrb.utils.parse_data import load_avro

In [2]:
logging.basicConfig(level="DEBUG")

In [3]:
pd.options.mode.copy_on_write = True

In [4]:
# path = Path.home().joinpath("Data/candidates_test.csv")
path = Path.home().joinpath("Data/local_data_candidates.csv")
print(f"Path is {path}")
assert path.exists(), "Failed to find path"

Path is /Users/robertstein/Data/local_data_candidates.csv


In [5]:
df = pd.read_csv(path)

In [6]:
print([x for x in df.columns])

['candid', 'objectid', 'deprecated', 'jd', 'utctime', 'sourceid', 'diffid', 'stackid', 'fid', 'exptime', 'progname', 'isdiffpos', 'fieldid', 'ra', 'dec', 'magzpsci', 'magzpsciunc', 'diffmaglim', 'magpsf', 'sigmapsf', 'chipsf', 'magap', 'sigmagap', 'magapbig', 'sigmagapbig', 'magdiff', 'magfromlim', 'distnr', 'magnr', 'sigmagnr', 'xpos', 'ypos', 'sky', 'fwhm', 'mindtoedge', 'seeratio', 'aimage', 'bimage', 'aimagerat', 'bimagerat', 'elong', 'nneg', 'nbad', 'sumrat', 'dsnrms', 'ssnrms', 'dsdiff', 'scorr', 'rb', 'rbversion', 'ssdistnr', 'ssmagnr', 'ssnamenr', 'tooflag', 'psobjectid1', 'sgmag1', 'srmag1', 'simag1', 'szmag1', 'distpsnr1', 'sgscore1', 'psobjectid2', 'sgmag2', 'srmag2', 'simag2', 'szmag2', 'distpsnr2', 'sgscore2', 'psobjectid3', 'sgmag3', 'srmag3', 'simag3', 'szmag3', 'distpsnr3', 'sgscore3', 'tmjmag1', 'tmhmag1', 'tmkmag1', 'tmobjectid1', 'tmjmag2', 'tmhmag2', 'tmkmag2', 'tmobjectid2', 'tmjmag3', 'tmhmag3', 'tmkmag3', 'tmobjectid3', 'neargaia', 'neargaiabright', 'maggaia', 'm

In [7]:
df

,candid,objectid,deprecated,jd,utctime,sourceid,diffid,stackid,fid,exptime,...,tmkmag2,tmobjectid2,tmjmag3,tmhmag3,tmkmag3,tmobjectid3,neargaia,neargaiabright,maggaia,maggaiabright
0,48229,WNTR23aabys,False,2.460249e+06,2023-10-31 04:24:35.214-07,139650,65,10784307506,2,900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48080,WNTR23aabwd,True,2.460153e+06,2023-07-27 03:53:57.778-07,138215,3,9954683703,2,600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Load cutouts
for diffid in tqdm(sorted(set(df["diffid"]))[:1]):

    print(diffid)
    df_cut = df[df["diffid"] == diffid]

    dbconstraints = DBQueryConstraints()
    dbconstraints.add_constraint(
        column="diffid",
        accepted_values=diffid,
    )
    res = select_from_table(dbconstraints, Diff.sql_model, output_columns="savepath", max_num_results=1)
    if len(res) > 0:
        diff_path = Path(res.iloc[0])
        avro_path = diff_path.parent.parent.joinpath(f"avro/{diff_path.with_suffix('.avro').name}")
        print(avro_path)
        if avro_path.exists():
            avro_df = load_avro(avro_path)
            # print(df_cut["candid"])
            for candid in df_cut["candid"]:
                print(candid, candid in avro_df["candid"])
            print(diffid, candid)

  0%|                                                                                                                                                     | 0/1 [00:00<?, ?it/s]

3
/Users/robertstein/Data/winter/20230726/avro/WINTERcamera_20230727-035357-778_mef_2_0_0_stack.resamp.diff.avro


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.29it/s]

48080 False
3 48080


In [9]:
avro_df["candid"]

0       75118
1       75119
2       75120
3       75121
4       75122
        ...  
1274    76392
1275    76393
1276    76394
1277    76395
1278    76396
Name: candid, Length: 1279, dtype: int64

In [10]:
# Export to Fritz
for utc_str in tqdm(sorted(set(df["utctime"]))):
    pipe = get_pipeline("WINTER", selected_configurations="name_candidates")

    dataset = Dataset()

    time = Time(utc_str[:-3].replace(" ", "T"), scale="utc", format='isot')

    df_sub = df[df["utctime"] == utc_str]

    print(time)

    metadata = {
        "RAWPATH": "history",
        "BASENAME": "history",
        "CALSTEPS": "",
        "DATE-OBS": time.isot,
        "utctime": time.isot,
        "diffmaglim": (df_sub["magpsf"] + df_sub["magfromlim"]).iloc[0],
        "jd": time.jd
    }
    df["DATE-OBS"] = time.isot
        
    source_table = SourceTable(source_list=df_sub, metadata=metadata)
    batch = SourceBatch()
    batch.append(source_table)
    dataset.append(batch)

    batches, errorstack = pipe.reduce_images(dataset=dataset, catch_all_errors=False, selected_configurations=["send_with_history"])

  0%|                                                                                                                                                     | 0/2 [00:00<?, ?it/s]INFO:mirar.pipelines:Found WINTER pipeline
INFO:mirar.pipelines.base_pipeline:Using pipeline configuration send_with_history (1/1)
DEBUG:mirar.pipelines.base_pipeline:Setting pipeline configuration to send_with_history.
DEBUG:mirar.pipelines.base_pipeline:Initialising processor <class 'mirar.processors.database.database_selector.SelectSourcesWithMetadata'>
DEBUG:mirar.pipelines.base_pipeline:Initialising processor <class 'mirar.processors.sources.source_table_modifier.CustomSourceTableModifier'>
DEBUG:mirar.pipelines.base_pipeline:Initialising processor <class 'mirar.processors.skyportal.skyportal_candidate.SkyportalCandidateUploader'>
DEBUG:mirar.pipelines.base_pipeline:Pipeline initialisation complete.
INFO:mirar.pipelines.base_pipeline:Applying '<class 'mirar.processors.database.database_selector.SelectSources

2023-07-27T03:53:57.778


  0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:mirar.processors.database.database_selector:Query constraints: sourceid = '138215'
DEBUG:mirar.processors.database.database_selector:Query constraints: sourceid = '138215' AND deprecated = 'False'
INFO:mirar.pipelines.base_pipeline:Applying '<class 'mirar.processors.sources.source_table_modifier.CustomSourceTableModifier'> to 1 batches (Step 2/3)
INFO:mirar.pipelines.base_pipeline:[Processor to modify a source dataframe using the winter_skyportal_annotator function.]
INFO:mirar.processors.base_processor:Running CustomSourceTableModifier on 1 threads


  0%|          | 0/1 [00:00<?, ?it/s]

...

DEBUG:mirar.processors.database.database_selector:Query constraints: sourceid = '139650'
DEBUG:mirar.processors.database.database_selector:Query constraints: sourceid = '139650' AND deprecated = 'False'
INFO:mirar.pipelines.base_pipeline:Applying '<class 'mirar.processors.sources.source_table_modifier.CustomSourceTableModifier'> to 1 batches (Step 2/3)
INFO:mirar.pipelines.base_pipeline:[Processor to modify a source dataframe using the winter_skyportal_annotator function.]
INFO:mirar.processors.base_processor:Running CustomSourceTableModifier on 1 threads


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:mirar.pipelines.base_pipeline:Applying '<class 'mirar.processors.skyportal.skyportal_candidate.SkyportalCandidateUploader'> to 1 batches (Step 3/3)
INFO:mirar.pipelines.base_pipeline:[<mirar.processors.skyportal.skyportal_candidate.SkyportalCandidateUploader object at 0x16b12b350>]
INFO:mirar.processors.base_processor:Running SkyportalCandidateUploader on 1 threads


  0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:mirar.processors.skyportal.skyportal_candidate:Checking if WNTR23aabys is candidate in SkyPortal
DEBUG:urllib3.connectionpool:https://fritz.science:443 "HEAD /api/candidates/WNTR23aabys HTTP/1.1" 200 0
DEBUG:mirar.processors.skyportal.skyportal_candidate:WNTR23aabys is candidate in SkyPortal
DEBUG:mirar.processors.skyportal.skyportal_candidate:Checking if WNTR23aabys is source in SkyPortal
DEBUG:urllib3.connectionpool:https://fritz.science:443 "HEAD /api/sources/WNTR23aabys HTTP/1.1" 200 0
DEBUG:mirar.processors.skyportal.skyportal_candidate:WNTR23aabys is source in SkyPortal
DEBUG:mirar.processors.skyportal.skyportal_candidate:Posting metadata of WNTR23aabys 48229 to SkyPortal
DEBUG:urllib3.connectionpool:https://fritz.science:443 "POST /api/candidates HTTP/1.1" 200 136
DEBUG:mirar.processors.skyportal.skyportal_candidate:Posted WNTR23aabys 48229 metadata to SkyPortal
DEBUG:urllib3.connectionpool:https://fritz.science:443 "GET /api/sources/WNTR23aabys/annotations HTTP/1.1" 200 4